In [16]:
import pandas
import folium
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import geocoder
import plotly.express as px

In [17]:
data = pandas.read_csv('sentiment_final.csv')

In [18]:
data = data.set_index('Unnamed: 0')

In [19]:
data

,date,Hiker trail name,Hiker Journal Link,Journal Story,Start location,Destination,Today Miles,Latitude,Longitude,State,Total Shelters,Occurrence,year,month,label,Emotion_scores,state,club,Acronym
Unnamed: 0,,,,,,,,,,,,,,,,,,,
5.0,1/1/2016 0:00,Hestia,https://www.trailjournals.com//journal/entry/5...,Woke up to 32F. Annie made coffee and we met P...,Hawk Mtn. Shelter\n,gooch mt shelter,7.3,34.655694,-84.049957,Georgia,60.0,4.0,2016.0,1.0,neutral,"[{'label': 'anger', 'score': 0.004779126960784...",GA,Georgia Appalachian Trail Club,GATC
6.0,1/1/2016 0:00,Brighty and Wreck,https://www.trailjournals.com//journal/entry/5...,Happy New Year!!! Our next to last section of ...,Ashby Gap\n,trico tower,7.2,44.525200,-70.980900,NaN,29.0,1.0,2016.0,1.0,joy,"[{'label': 'anger', 'score': 0.002314480487257...",ME,Appalachian Mountain Club,AMC
7.0,1/2/2016 0:00,Brother Blood,https://www.trailjournals.com//journal/entry/5...,"It's 2016, and the past two weeks have been mo...",NaN,its 2016,0.0,37.817700,-79.070600,NaN,20.0,1.0,2016.0,1.0,neutral,"[{'label': 'anger', 'score': 0.006532554049044...",VA,Tidewater Appalachian Trail Club,TATC
10.0,1/3/2016 0:00,Hestia,https://www.trailjournals.com//journal/entry/5...,Wow. Yesterday was crazy! I started my period ...,Blood Mtn Shelter\n,mountain crossings hostel,1.5,36.809400,-81.422600,NaN,60.0,1.0,2016.0,1.0,sadness,NaN,VA,Piedmont Appalachian Trail Hikers,PATH
11.0,1/3/2016 0:00,SIMPLE MAN,https://www.trailjournals.com//journal/entry/5...,"Hi,my name is Billy,trail name ""SIMPLE MAN"".I ...",NaN,maine,0.0,40.839400,-75.359800,NaN,0.0,17.0,2016.0,1.0,sadness,"[{'label': 'anger', 'score': 0.002191700274124...",PA,AMC Delaware Valley Chapter,AMC-DV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33192.0,10/12/2023 0:00,Flippertree for Autism,https://www.trailjournals.com//journal/entry/4...,Our 2014 goal of getting to Swatara Gap is com...,Route 183\n,rt 501,9.0,40.512510,-76.346900,NaN,1.0,1.0,2023.0,10.0,neutral,"[{'label': 'anger', 'score': 0.012704795226454...",PA,Blue Mountain Eagle Climbing Club,BMECC
33212.0,11/9/2023 0:00,Flippertree for Autism,https://www.trailjournals.com//journal/entry/6...,Nick had a rough night in hotel. I guess he w...,Neighbor Mountain Parking\n,hogback,13.0,35.963950,-82.587200,NaN,1.0,1.0,2023.0,11.0,fear,"[{'label': 'anger', 'score': 0.009218733757734...",NC,Carolina Mountain Club,CMC
33224.0,12/5/2023 0:00,Flippertree for Autism,https://www.trailjournals.com//journal/entry/5...,It was Tyler's Birthday and he wanted money to...,Route 225 Pedestrian Bridge Parking Lot\n,clarks ferry bridge,6.7,40.392220,-76.994000,NaN,1.0,1.0,2023.0,12.0,neutral,"[{'label': 'anger', 'score': 0.014029124751687...",PA,NaN,NaN


## Folium

In [20]:
# t = data[data['year'] == 2016]
# t = t[t['label'] == 'joy']

In [21]:
# #color = ['blue', 'orange', 'purple', 'green', 'pink', 'cyan', 'lime', 'yellow', 'red', 'navy', 'brown']

# #pin_color = ['darkblue', 'orange', 'purple', 'green', 'pink', 'lightblue', 'lightgreen', 'beige', 'red']
# map = folium.Map(location=[38, -84], zoom_start=6, min_zoom=5, max_bounds=True)
# for index in range(len(t)):
#     df = t.iloc[index]
#     #popup = folium.Popup(htmldiplay(data_clean[data_clean['beachid'] == index]), max_width=900)
#     marker = folium.Marker([df['Latitude'], df['Longitude']])
#     marker.add_to(map)

# map

## Plotly: Emotion Distribution over States

In [31]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)


app.layout = html.Div([
    html.H4('Emotion distribution over States'),
    html.P("Select an Emotion:"),
    dcc.RadioItems(
        id='emotion', 
        options=[{'label': emotion, 'value': emotion} for emotion in ['neutral', 'joy', 'sadness', 'surprise', 'fear', 'anger', 'disgust']],
        value="neutral",
        inline=True
    ),
    html.P("Select a Year:"),
    dcc.Slider(
        id='year-slider',
        min=2016,
        max=2023,
        value=2023,
        marks={str(year): str(year) for year in range(2016, 2024)},
        step=None
    ),
    html.P("Select a Month:"),
    dcc.Slider(
        id='month-slider',
        min=1,
        max=12,
        value=1,
        marks={str(month): str(month) for month in range(1, 13)},
        step=None
    ),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output("graph", "figure"), 
    [Input("emotion", "value"),
     Input("year-slider", "value"),
     Input("month-slider", "value")]
)
def display_choropleth(emotion, year, month):
    df = data[(data['label'] == emotion) & (data['year'] == year) & (data['month'] == month)]['state'].value_counts().reset_index()
    fig = px.choropleth(
        df,
        locationmode="USA-states",
        locations='state',
        color='count',
        scope="usa",
        color_continuous_scale="OrRd",
        title='Emotion distribution over States'
    )
    #fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig


app.run_server(debug=True)

## Ploty: Emotion Distribution over Clubs

In [23]:
# g = pandas.read_excel("Trail Clubs' Geo-location data.xlsx")

In [24]:
# g

In [25]:
# def get_label_for_coordinate(lat, lon, df):
#     """
#     Get the label for a coordinate if it falls into any box defined in the dataframe.

#     Parameters:
#     - lat (float): Latitude of the point.
#     - lon (float): Longitude of the point.
#     - df (pd.DataFrame): Dataframe containing the box coordinates and labels.

#     Returns:
#     - str: Label of the box the point falls into or None if the point is outside all boxes.
#     """
    
#     for index, row in df.iterrows():
#         if (row['latitude_start'] <= lat <= row['latitude_end']) and (row['longitude_start'] <= lon <= row['longitude_end']):
#             return row['label']
    
#     return pandas.NA

In [26]:
# get_label_for_coordinate(35.042000,-83.548100, g)

In [27]:
# data[data['club'].isna()]

In [28]:
import seaborn as sns

In [32]:
app = Dash(__name__)


app.layout = html.Div([
    html.H4('Emotion distribution over Clubs'),
    html.P("Select an Emotion:"),
    dcc.RadioItems(
        id='emotion', 
        options=[{'label': emotion, 'value': emotion} for emotion in ['neutral', 'joy', 'sadness', 'surprise', 'fear', 'anger', 'disgust']],
        value="neutral",
        inline=True
    ),
    html.P("Select a Year:"),
    dcc.Slider(
        id='year-slider',
        min=2016,
        max=2023,
        value=2023,
        marks={str(year): str(year) for year in range(2016, 2024)},
        step=None
    ),
    html.P("Select a Month:"),
    dcc.Slider(
        id='month-slider',
        min=1,
        max=12,
        value=1,
        marks={str(month): str(month) for month in range(1, 13)},
        step=None
    ),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output("graph", "figure"), 
    [Input("emotion", "value"),
     Input("year-slider", "value"),
     Input("month-slider", "value")]
)
def display_choropleth(emotion, year, month):
    df = data[(data['label'] == emotion) & (data['year'] == year) & (data['month'] == month)]

    # Use seaborn to generate a palette of 24 colors
    colors = sns.color_palette("husl", 24).as_hex()

    # Create a color map for each unique club
    clubs = df['club'].unique()
    if len(clubs) > len(colors):
        raise ValueError("The number of unique clubs exceeds available colors.")

    color_map = dict(zip(clubs, colors))
    
    # Create an empty figure
    fig = go.Figure()

    # Add a trace for each club
    for club in clubs:
        subset = df[df['club'] == club]
        fig.add_trace(go.Scattergeo(
            lon = subset['Longitude'],
            lat = subset['Latitude'],
            text = subset['label'],
            mode = 'markers',
            marker = dict(
                color = color_map[club],
                size = 10,
                opacity = 0.8
            ),
            name = club,
            showlegend=True
        ))

    fig.update_layout(
            title = 'Emotion distribution over Clubs',
            geo_scope='usa',
        )

    return fig


app.run_server(debug=True)

In [30]:
data

,date,Hiker trail name,Hiker Journal Link,Journal Story,Start location,Destination,Today Miles,Latitude,Longitude,State,Total Shelters,Occurrence,year,month,label,Emotion_scores,state,club,Acronym
Unnamed: 0,,,,,,,,,,,,,,,,,,,
5.0,1/1/2016 0:00,Hestia,https://www.trailjournals.com//journal/entry/5...,Woke up to 32F. Annie made coffee and we met P...,Hawk Mtn. Shelter\n,gooch mt shelter,7.3,34.655694,-84.049957,Georgia,60.0,4.0,2016.0,1.0,neutral,"[{'label': 'anger', 'score': 0.004779126960784...",GA,Georgia Appalachian Trail Club,GATC
6.0,1/1/2016 0:00,Brighty and Wreck,https://www.trailjournals.com//journal/entry/5...,Happy New Year!!! Our next to last section of ...,Ashby Gap\n,trico tower,7.2,44.525200,-70.980900,NaN,29.0,1.0,2016.0,1.0,joy,"[{'label': 'anger', 'score': 0.002314480487257...",ME,Appalachian Mountain Club,AMC
7.0,1/2/2016 0:00,Brother Blood,https://www.trailjournals.com//journal/entry/5...,"It's 2016, and the past two weeks have been mo...",NaN,its 2016,0.0,37.817700,-79.070600,NaN,20.0,1.0,2016.0,1.0,neutral,"[{'label': 'anger', 'score': 0.006532554049044...",VA,Tidewater Appalachian Trail Club,TATC
10.0,1/3/2016 0:00,Hestia,https://www.trailjournals.com//journal/entry/5...,Wow. Yesterday was crazy! I started my period ...,Blood Mtn Shelter\n,mountain crossings hostel,1.5,36.809400,-81.422600,NaN,60.0,1.0,2016.0,1.0,sadness,NaN,VA,Piedmont Appalachian Trail Hikers,PATH
11.0,1/3/2016 0:00,SIMPLE MAN,https://www.trailjournals.com//journal/entry/5...,"Hi,my name is Billy,trail name ""SIMPLE MAN"".I ...",NaN,maine,0.0,40.839400,-75.359800,NaN,0.0,17.0,2016.0,1.0,sadness,"[{'label': 'anger', 'score': 0.002191700274124...",PA,AMC Delaware Valley Chapter,AMC-DV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33192.0,10/12/2023 0:00,Flippertree for Autism,https://www.trailjournals.com//journal/entry/4...,Our 2014 goal of getting to Swatara Gap is com...,Route 183\n,rt 501,9.0,40.512510,-76.346900,NaN,1.0,1.0,2023.0,10.0,neutral,"[{'label': 'anger', 'score': 0.012704795226454...",PA,Blue Mountain Eagle Climbing Club,BMECC
33212.0,11/9/2023 0:00,Flippertree for Autism,https://www.trailjournals.com//journal/entry/6...,Nick had a rough night in hotel. I guess he w...,Neighbor Mountain Parking\n,hogback,13.0,35.963950,-82.587200,NaN,1.0,1.0,2023.0,11.0,fear,"[{'label': 'anger', 'score': 0.009218733757734...",NC,Carolina Mountain Club,CMC
33224.0,12/5/2023 0:00,Flippertree for Autism,https://www.trailjournals.com//journal/entry/5...,It was Tyler's Birthday and he wanted money to...,Route 225 Pedestrian Bridge Parking Lot\n,clarks ferry bridge,6.7,40.392220,-76.994000,NaN,1.0,1.0,2023.0,12.0,neutral,"[{'label': 'anger', 'score': 0.014029124751687...",PA,NaN,NaN
